# SQLAlchemy and vector search (2 points)

For defining the table, we will use Python and SQLAlchemy framework.

A good practice is to build the database URL using the SQLAlchemy library. This option is much more readable, safer, and easier to maintain than using the connection string directly.

In [1]:
from sqlalchemy.engine import URL

db_url = URL.create(
    drivername="postgresql+psycopg",
    username="postgres",
    password="password",
    host="localhost",
    port=5556,
    database="similarity_search_service_db"
)

Tables in SQLAlchemy are defined using class-based design. Typically, application defines a single Base class, from which concrete tables inherit. They are defined quite similarly to Pydantic, i.e. with attributes and types. However, here we also need to assign concrete database types. To integrate it with vector search, we will also use pgvector library.

In [2]:
from pgvector.sqlalchemy import Vector
from typing import List
from sqlalchemy import Integer, String #, List
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column


# Create the base class for the table definition
class Base(DeclarativeBase):
    __abstract__ = True


# Create the table definition
class Images(Base):
    __tablename__ = "images"
    VECTOR_LENGTH = 512
    
    # primary key
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    # image path - we will use it to store the path to the image file, after similarity search we can use it to retrieve the image and display it
    image_path: Mapped[str] = mapped_column(String(256))
    # image embedding - we will store the image embedding in this column, the image embedding is a list of 512 floats this is the output of the sentence transformer model
    image_embedding: Mapped[List[float]] = mapped_column(Vector(VECTOR_LENGTH))

To actually connect to the database, interact with it and run queries, we use engine object. It is created with create_engine() function.

In [3]:
from sqlalchemy import create_engine

engine = create_engine(db_url)

Now we can create the table in the database.

In [4]:
Base.metadata.create_all(engine)

# SQLAlchemy queries

For testing, we need to insert some data into the table. Code below has one thing missing - you need to create the Images object based on provided data.

In [5]:
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import select
import numpy as np

# reusable function to insert data into the table
def insert_image(engine: sqlalchemy.Engine, image_path: str, image_embedding: list[float]):
    with Session(engine) as session:
        # create the image object
        image = Images(image_path=image_path, image_embedding=image_embedding)
        # add the image object to the session
        session.add(image)
        # commit the transaction
        session.commit()

# calculate the cosine similarity between the first image and the K rest of the images, order the images by the similarity score
def find_k_images(engine: sqlalchemy.Engine, k: int, orginal_image: Images) -> list[Images]:
    with Session(engine) as session:
        # execution_options={"prebuffer_rows": True} is used to prebuffer the rows, this is useful when we want to fetch the rows in chunks and return them after session is closed
        result = session.execute(
            select(Images)
            .order_by(Images.image_embedding.cosine_distance(orginal_image.image_embedding))
            .limit(k), 
            execution_options={"prebuffer_rows": True}
        )
        return result

# insert some data into the table
N = 100
for i in range(N):
    image_path = f"image_{i}.jpg"
    image_embedding = np.random.rand(512).tolist()
    insert_image(engine, image_path, image_embedding)

# select first image from the table
with Session(engine) as session:
    image = session.query(Images).first()

# find the 10 most similar images to the first image
k = 10
similar_images = find_k_images(engine, k, image)

In [6]:
for img in similar_images:
    print(img[0].image_path)

image_0.jpg
image_42.jpg
image_89.jpg
image_22.jpg
image_38.jpg
image_81.jpg
image_60.jpg
image_55.jpg
image_93.jpg
image_6.jpg


# Filtering
We can filter the found nearest neighbors, e.g. by required minimal similarity score.

In [7]:
# find the images with the similarity score greater than 0.9
def find_images_with_similarity_score_greater_than(engine: sqlalchemy.Engine, similarity_score: float, orginal_image: Images) -> list[Images]:
    with Session(engine) as session:
        result = session.execute(
            select(Images)
            .filter(Image.image_embedding.cosine_distance(orginal_image.image_embedding) > similarity_score), 
            execution_options={"prebuffer_rows": True}
        )
        return result

For more filtering, we will need an actual dataset with rich metadata to filter by. For this, we will utilize Steam Games Dataset. It is hosted on HuggingFace Hub, and we can download it with:

In [8]:
from datasets import load_dataset

dataset = load_dataset("FronkonGames/steam-games-dataset")

# get columns names and types
columns = dataset["train"].features
print(columns)

columns_to_keep = ["Name", "Windows", "Linux", "Mac", "About the game", "Supported languages", "Price"]

N = 40000
dataset = dataset["train"].select_columns(columns_to_keep).select(range(N))

{'AppID': Value('int64'), 'Name': Value('string'), 'Release date': Value('string'), 'Estimated owners': Value('string'), 'Peak CCU': Value('int64'), 'Required age': Value('int64'), 'Price': Value('float64'), 'DLC count': Value('int64'), 'About the game': Value('string'), 'Supported languages': Value('string'), 'Full audio languages': Value('string'), 'Reviews': Value('string'), 'Header image': Value('string'), 'Website': Value('string'), 'Support url': Value('string'), 'Support email': Value('string'), 'Windows': Value('bool'), 'Mac': Value('bool'), 'Linux': Value('bool'), 'Metacritic score': Value('int64'), 'Metacritic url': Value('string'), 'User score': Value('int64'), 'Positive': Value('int64'), 'Negative': Value('int64'), 'Score rank': Value('float64'), 'Achievements': Value('int64'), 'Recommendations': Value('int64'), 'Notes': Value('string'), 'Average playtime forever': Value('int64'), 'Average playtime two weeks': Value('int64'), 'Median playtime forever': Value('int64'), 'Medi

We will use columns:
- Name
- About the game
- Price
- Platforms - the platforms on which the game is available; note that there is separate field for each platform (Windows, Linux, macOS)

For vector search, we can use the About the game column, which is an arbitrary text description. A great model for this purpose is distiluse-base-multilingual-cased-v2 from Sentence Transformers. This model is a multilingual text transformers, and thus it will work well for descriptions of games in languages other than English.

The distiluse-base-multilingual-cased-v2 model supports over 50 languages. This version is a distilled multilingual knowledge model derived from the original Universal Sentence Encoder, which only supported 15 languages. While the v2 model supports a wider range of languages, it is noted that its performance may be a bit lower, compared to the original model for the languages it shares with v1, particularly the first 15 languages that were supported originally. Read the paper if you're interested: https://arxiv.org/pdf/2004.09813.

When looking at https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2, one can see that it produces 512-dimensional embeddings. This is a value we need to declare in the table definition as vector length.

In [9]:
from sqlalchemy import Integer, Float, Boolean


class Games(Base):
    __tablename__ = "games"
    __table_args__ = {'extend_existing': True}
    
    # the vector size produced by the model taken from documentation https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2
    VECTOR_LENGTH = 512 # check the model output dimensionality 
        
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String(256))
    description: Mapped[str] = mapped_column(String(4096))
    windows: Mapped[bool] = mapped_column(Boolean)
    linux: Mapped[bool] = mapped_column(Boolean)
    mac: Mapped[bool] = mapped_column(Boolean)
    price: Mapped[float] = mapped_column(Float)
    game_description_embedding: Mapped[List[int]] = mapped_column(Vector(VECTOR_LENGTH)) # fill it in proper way

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

Let's prepare the function that will generate the embeddings for the games descriptions.

In [10]:
from sentence_transformers import SentenceTransformer
import torch


checkpoint = "distiluse-base-multilingual-cased-v2"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer(checkpoint, device=device)


def generate_embeddings(text: str) -> list[float]:
    return model.encode(text)

def generate_embeddings_batch(texts):
    return model.encode(texts, batch_size=32, show_progress_bar=False, device='cuda')

In [11]:
model.device

device(type='cuda', index=0)

Now let's prepare the function that will insert the data into the table. (I've also changed it a little to handle batches instead of single points to make it faster and more gpu friendly (6min down to 6 sec :> )

In [12]:
from tqdm import tqdm
import gc

def insert_games(engine, dataset, batch_size=1000):
    buffer = []
    descriptions = []

    with tqdm(total=len(dataset)) as pbar:
        for i, game in enumerate(dataset):
            game_description = game["About the game"] or ""
            name, windows, linux, mac, price = game["Name"], game["Windows"], game["Linux"], game["Mac"], game["Price"]

            if name and windows and linux and mac and price and game_description:
                descriptions.append(game_description[:4096])
                buffer.append(game)

            if len(buffer) >= batch_size:
                embeddings = model.encode(descriptions, batch_size=batch_size, show_progress_bar=False, device='cuda')
                with Session(engine) as session:
                    for g, emb, desc in zip(buffer, embeddings, descriptions):
                        game_obj = Games(
                            name=g["Name"],
                            description=desc,
                            windows=g["Windows"],
                            linux=g["Linux"],
                            mac=g["Mac"],
                            price=g["Price"],
                            game_description_embedding=emb.tolist(),
                        )
                        session.add(game_obj)
                    session.commit()
                buffer.clear()
                descriptions.clear()

            pbar.update(1)

        if buffer:
            embeddings = model.encode(descriptions, batch_size=batch_size, show_progress_bar=False, device='cuda')
            with Session(engine) as session:
                for g, emb, desc in zip(buffer, embeddings, descriptions):
                    game_obj = Games(
                        name=g["Name"],
                        description=desc,
                        windows=g["Windows"],
                        linux=g["Linux"],
                        mac=g["Mac"],
                        price=g["Price"],
                        game_description_embedding=emb.tolist(),
                    )
                    session.add(game_obj)
                session.commit()
            buffer.clear()
            descriptions.clear()
    gc.collect()

In [13]:
insert_games(engine, dataset)

100%|██████████████████████████████████████████████████████████████████████████████| 40000/40000 [00:06<00:00, 6624.96it/s]


Now the function that will find the games similar to the given game, and also include given filtering criteria.

In [16]:
from typing import Optional

def find_game(
    engine: sqlalchemy.Engine, 
    game_description: str, 
    windows: Optional[bool] = None, 
    linux: Optional[bool] = None,
    mac: Optional[bool] = None,
    price: Optional[int] = None
):
    with Session(engine) as session:
        game_embedding = generate_embeddings(game_description)
    
        query = (
            select(Games)
            .order_by(Games.game_description_embedding.cosine_distance(game_embedding))
        )
        
        if price:
            query = query.filter(Games.price <= price)
        if windows:
            query = query.filter(Games.windows == True)
        if linux:
            query = query.filter(Games.linux == True)
        if mac:
            query = query.filter(Games.mac == True)
        
        result = session.execute(query, execution_options={"prebuffer_rows": True})
        game = result.scalars().first()
        
        return game

Our first vector search service is ready to use! Let's check it out.

In [17]:
game = find_game(engine, "This is a game about a hero who saves the world", price=10)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

game = find_game(engine, game_description="Home decorating", price=20)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

Game: Ultimate Spider Hero
Description: Ultimate Spider Hero game was designed for real heroes! Your mission is to help poor residents of the Metropolis and to save them from the terrible monsters. Move forward to fight your enemies and try not to fall! Features: Simple and addictive gameplay Nice graphics Awesome Ultimate Spider Hero Countless Steam achievements for you to collect! Compatibility with multiple major platforms (Windows, Mac, Linux, SteamOS) Make your way through the endless labyrinths of long, confusing city streets together with your favorite hero from countless movies and cartoons! Although this may look simple enough, things are not as easy as they seem. You will have to learn how to cling into houses properly using your web, otherwise you will fall to your demise. If you manage to do so - you will become a real superhero, armed with elusiveness, agility and speed and the ability to tirelessly swing across the rooftops and between the huge skyscrapers this urban land

Let's change the filtering requirements:

In [18]:
game = find_game(engine, game_description="Home decorating", mac=True, price=5)
print(f"Game: {game.name}")
print(f"Description: {game.description}")

Game: 3D PUZZLE - Old House
Description: Collect a 3D puzzle, transferring things to the right places to create a beautiful house. You need to go to the item, take it by pressing the left mouse button and take the item to the desired location marked in green. If you brought the correct item, it will snap into place and you will receive leaderboard points and achievements for this. Collect as much substance as possible as quickly as possible to get more points for the leaderboard. If you brought the wrong item, you can throw it away, it will return to the starting location so that you can pick it up again.


As you can see, while changing criteria to more strict, results can vary. This is the consequence of a few things: how attribute filtering reduces the results, how model interprets the similarity of descriptions, and how those two things interact together. A major advantage of this approach is its overall simplicity.